In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install albumentations

     |████████████████████████████████| 634kB 5.3MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654021 sha256=b9275167d4dd70a387dffec9087034afe07fb4886a6b7138d177b9fc32cca750
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [21]:
import random
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [4]:
import os
os.chdir("/content/drive/My Drive/")
!pwd

/content/drive/My Drive


In [14]:
PATH = Path("/content/drive/My Drive/VisionX/data/images/")

In [9]:
train_df = pd.read_csv("/content/drive/My Drive/VisionX/train.csv")

In [10]:
print('healthy',len(train_df[train_df['healthy']==1]))
print('multiple_diseases',len(train_df[train_df['multiple_diseases']==1]))
print('rust',len(train_df[train_df['rust']==1]))
print('scab',len(train_df[train_df['scab']==1]))

healthy 516
multiple_diseases 91
rust 622
scab 592


In [11]:
doc_aug = A.Compose([
  A.Transpose(),  
  A.Rotate(limit=30, p=0.5),
  A.HorizontalFlip(p=0.5),  
  A.VerticalFlip(p=0.5),  
  A.GaussNoise(p=0.25),
  A.RandomBrightnessContrast(p=0.2),
  ])
random.seed(42)

In [12]:
train_df[train_df['multiple_diseases']==1]['image_id']

1          Train_1
6          Train_6
25        Train_25
36        Train_36
62        Train_62
           ...    
1708    Train_1708
1779    Train_1779
1780    Train_1780
1784    Train_1784
1815    Train_1815
Name: image_id, Length: 91, dtype: object

In [13]:
df=pd.DataFrame(columns=['image_id','healthy','multiple_diseases','rust','scab'])

In [23]:

mlp_image=[]
for name in train_df[train_df['multiple_diseases']==1]['image_id']:
    path="/content/drive/My Drive/VisionX/data/images/"+ name+".jpg"
    img=cv2.imread(path)
    image=cv2.resize(img,(256,256),interpolation=cv2.INTER_AREA)
    for i in tqdm(range(5)):
        augmented = doc_aug(image=image)
        file_name = f'{name}_{i}'
        df=df.append({'image_id' : file_name , 'healthy' : 0,'multiple_diseases':1, 'rust':0 ,'scab': 0} , ignore_index=True)
        cv2.imwrite(f'{PATH}/{file_name}.jpg',augmented['image'])
    

100%|██████████| 5/5 [00:00<00:00, 80.00it/s]


In [24]:
train_df0=pd.concat([train_df,df],axis=0,ignore_index=True)

In [25]:
shuffled_train=train_df0.sample(frac=1)

In [26]:
shuffled_train.to_csv(r'shuffled_train.csv',index=False)